### Loading  Training and Testing Dataset

In [1]:
import pandas as pd

Testing:
- polarity : Target 2:positive, 1:neutral, 0:negative
- words : preprocessed sentences
- type : the tags of the words from lemmatizing 

In [3]:
testing = pd.read_csv('./clean_data/testing_data.csv', encoding='utf8')
print testing.isnull().sum()
testing[testing.words.isnull()]
training = pd.read_csv('./clean_data/training_data.csv', encoding='utf8')
print training.isnull().sum()
training[training.words.isnull()]

words    1
type     1
dtype: int64
words       1
type        1
polarity    0
dtype: int64


,words,type,polarity
747,NaN,NaN,2.0


Training:
- polarity : Target 2:positive, 1:neutral, 0:negative
- words : preprocessed sentences
- type : the tags of the words from lemmatizing 

As I was loading the data I found that there were null values appearing. When referring back, these values came from the cleaning process and were converted to Null values when they were saved to csv. 

In [5]:
testing = testing.dropna()
training = training.dropna()
print training.isnull().sum()
print testing.isnull().sum()

words       0
type        0
polarity    0
dtype: int64
words    0
type     0
dtype: int64


In [6]:
training.head()

,words,type,polarity
0,judge previous post used be good place not longer,VB JJ NN VB VB JJ NN RB JJ,0.0
1,be arrive noon place be empty staff act be imp...,VB VB NN NN VB JJ NN VB VB VB VB RB JJ,0.0
2,never bring complimentary noodle ignore repeat...,RB VB JJ NN VB JJ NN NN VB NN NN,0.0
3,food be lousy too sweet too salty portion tiny,NN VB JJ RB JJ RB JJ NN JJ,0.0
4,food be lousy too sweet too salty portion tiny,NN VB JJ RB JJ RB JJ NN JJ,0.0


## Train Test Split 
- The use of a Train Test Split is to test how well our classifier is working for predicitons.

https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6 

------ look up stratified...

In [7]:
from sklearn.model_selection import train_test_split

# predictor
X = training.words
# target
y = training.polarity
#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

### Baseline of Model
- The baseline accuracy is the proportion of the majority class. In this case '2' which is positive sentiment and so the baseline accuracy is 0.724138

baseline_accuracy = majority class N / total N


In [8]:
print y.value_counts(normalize=True)

baseline = 0.724138

2.0    0.724138
0.0    0.243799
1.0    0.032063
Name: polarity, dtype: float64


## Feature Transformation for Training Data
- Count Vectorizer and TFIDF Vectorizer are both are used to compare the best fitting thus, choosing which transformer is best aporopriate.
- Logistic Regression is the classifier used to measure this. 
Logistic regression is a very fast classifier???


In [9]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score

In [10]:
lr = LogisticRegression(random_state=1)


### Count Vectorizer on Training

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
# initalise the vectoriser 
cvec = CountVectorizer()
# fit the training data on the model
cvec.fit(X_train)

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [12]:
#transform training data into sparse matrix
X_train_cvec = cvec.transform(X_train)
X_test_cvec = cvec.transform(X_test)

# fit with Logistic Regression
lr.fit(X_train_cvec, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
# predict with LR
y_pred_cvec = lr.predict(X_test_cvec)
cvec_score  = accuracy_score(y_test, y_pred_cvec)

We still have the same number of rows but the vectorization has converted every word, or what is believed to be a word, from our test data into a feature. This is like dummy coded variables for words except that we have counts rather than just occurances.???.... featured names of wueds frequency of the top highest words. 

#### COUNT VEC - Word Frequency 
matrix outputting the 10 most common words and how many times they appear 
- Feature matrix of word occurences 
- top 10 word most occuring words
- most important words

In [ ]:
df_cvec = pd.DataFrame(cvec_mat.todense(),columns=cvec.get_feature_names())
word_freq = df_cvec.sum(axis=0).sort_values(ascending=False)[:10]
word_freq

#### COUNT VEC - Zipf's law
It state that 
In a corpus of text, any words frequency is inversely proportional to its rank in the frequency table. 

Thus the most frequent word will occur approximately twice as often as the second most frequent word, three times as often as the third most frequent word, etc.: the rank-frequency distribution is an inverse relation


Unsurprisingly, the word frequencies for the collected works of Alexandre Dumas appear to follow the predictions made by Zipf’s law! In this blog post, I’ll show you how to use bash and R to perform this simple analysis.


Plotting the word frequency distribution, we observe : what is thissssssssss? https://en.wikipedia.org/wiki/Zipf%27s_law 

In [ ]:
# import matplotlib.pyplot as plt
# word_freq.plot(kind='hist',
#             title='Number of words with a given number of appearances',
#             fontsize=14)
# plt.show()

### TfIdF (term frequency inverse document frequency)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
# initalise the vectoriser 
tvec = TfidfVectorizer()
# fit the training data on the model
tvec.fit(X_train)

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [16]:
#transform training data into sparse matrix
X_train_tvec = tvec.transform(X_train)
X_test_tvec = tvec.transform(X_test)

# fit with Logistic Regression
lr.fit(X_train_tvec, y_train)

/Users/annabopeep/anaconda2/lib/python2.7/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [21]:
# predict with LR
y_pred_tvec = lr.predict(X_test_tvec)
tvec_score  = accuracy_score(y_test, y_pred_tvec)

#### Comparing scores
- Both vectorizers increased the accuracy
- TFIFD has the highest so is the chosen Feature Transformer that I will be choosing for my test set 

Hyperparameters will also be tested for the transformer
Both sparse matrices from the feature transformers (count vectoriser and TFIDF) 


In [22]:
print 'baseline:', baseline
print 'count vectorizer score:', cvec_score
print 'tfidf vectorizer score:', tvec_score

baseline: 0.724138
count vectorizer score: 0.7782258064516129
tfidf vectorizer score: 0.780241935483871


## TFIFD Vectorizer Parameters

TF-IDF (term frequency–inverse document frequency)is a calculation to transform words into numbers. It takes into account the frequecy of a word in a given document and the frequency between documents. So if a term is common in most documents it is supressed and rare words  are given more influence showing they are highly specific for a particular document. 

To break it down:
Term Frequency: More frequent terms in a target article, the higher the score
Inverse Document Frequency: The more common in other articles, the lower the score

Mathematically, the importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus.

TFcalculationformuala...
$$
\mathrm{idf}(t, D) = 1+\log\left(\frac{1+N_\text{Documents}}{1+N_\text{Documents that contain term}}\right)
$$


$$
\text{tf-idf}(t,d,D) = \mathrm{tf}(t,d) \cdot \mathrm{idf}(t, D)
$$

In [50]:
# new params for tfifd and fit to training
tvec_p = TfidfVectorizer(max_features=900, norm=None)
tvec_p.fit(X_train)

# transform to train and test 
X_train_tvec_p = tvec_p.transform(X_train)
X_test_tvec_p = tvec_p.transform(X_test)

# fit to LR, predict and score
lr.fit(X_train_tvec_p, y_train)
y_pred = lr.predict(X_test_tvec_p)
score = accuracy_score(y_test, y_pred)
print 'Baseline:', baseline
print 'Default Params:', tvec_score
print 'Optimal Params:', score
print "Number of features:", len(tvec.get_feature_names())

Baseline: 0.724138
Default Params: 0.780241935483871
Optimal Params: 0.7903225806451613
Number of features: 1626


In [ ]:
0.7883064516129032 max features
0.7903225806451613 + norm none


We choose parameters improve our analysis. There are some which can be applied but will not be used because the preprocessing step included them including stop words, lowercase and tokenizer. We will include parameters under a range which will be Grid Searched to give the optimal scores for the specific model in the pipeline. 

Parameters help show us how exactly we want to break down the document.

- max_features : 
The maxiumum number of highest term frequencies across the corpus will only be considered as the vocabulary. 
- norm : (‘l1’, ‘l2’, None)????
Norm used to normalize term vectors. None for no normalization.


- ngram_range :(min_n, max_n) i should only use this if on raw text????
The n-value range of lower and upper boundaries for n-grams to be extracted.
- min_df 
specifies that a word must be used at least twice to be considered. In practice this is useful for removing things like URLs from text, which appear as one offs.

- The max_df is a float value — this tell the vectorizer to ignore words which appears in more than 50% of documents in the corpus. This generally catches words not already defined in the stopwords set.

- We also define a set of stop words. These are words like “a” or “is” which appear so often in a language that we know they won’t provide useful information and so can be ignored.



http://apapiu.github.io/2016-08-04-tf_idf/

- Finally we use fit_transform() to train the vectorizer using the corpus we defined above.


sublinear_tf=True, max_features = 500 

max_features : int or None, default=None
    If not None, build a vocabulary that only consider the top
    max_features ordered by term frequency across the corpus.

    This parameter is ignored if vocabulary is not None.

sublinear_tf : boolean, default=False
    Apply sublinear tf scaling, i.e. replace tf with 1 + log(tf).
TF-IDF: combination of sublinear TF and inverse document frequency???

Term Frequency (TF)
local frequency of a word in the document
i.e. the word is weighed by how many times it occurs in the document
tf(w,d)=∣∣{w′∈d : w′=w}∣∣tf(w,d)=|{w′∈d : w′=w}| where ww is a word and d={w1, ... ,wm}d={w1, ... ,wm} is a document

Sublinear TF:
sometimes a word is used too often so we want to reduce its influence compared to other less frequently used words
for that we can use some sublinear function, e.g.
logtf(w,d)log⁡tf(w,d) or tf(w,d)‾‾‾‾‾‾‾√

http://0agr.ru/wiki/index.php/TF-IDF

# EDA

In [ ]:
tvec_df  = pd.DataFrame(tvec.transform(training.words).todense(),
                   columns=tvec.get_feature_names())
tvec_df.sum(axis=0).sort_values(ascending=False)[:10]

## Params on testing data
- TFDIF was fitted on the training data which will be used to transfomation the words in testing into a sparse matrix
- This will then use the logistic regressions best parameters which were fit for thr training data to predict sentiment/ y_hat for my testing data.


In [ ]:
testing.head(3)

In [ ]:
# transform the testing data
X_test_testing = tvec.transform(testing.words)

# predictions and predictive probabilities
y_hat = best_lr_tvec.predict(X_test_testing)
y_hat_pp = best_lr_tvec.predict_proba(X_test_testing)
y_hat_pp

logistic regression gave a good score in classifying the predictors but will try different classifiers to determine best accuracy score 

In [ ]:
best_lr_tvec

## next
- Should i try a different approach by grid searching the vectorisers to see which ones perform best rather than the classifier?? 
- next steps are to loop through a list of classifiers choose the best one/s performing then grid search in a pipline with the vectoriser to optimise both params